In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.utils import to_categorical
import time
import tensorflow as tf

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=3072)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

Virtual devices cannot be modified after being initialized


In [5]:
# Function to load preprocessed images and labels
def load_data(input_dirs):
    X = []
    y = []
    for i, input_dir in enumerate(input_dirs):
        for img_name in os.listdir(input_dir):
            img_path = os.path.join(input_dir, img_name)
            img = cv2.imread(img_path)
            X.append(img)
            y.append(i)
    return np.array(X), np.array(y)

In [6]:
# Data directories for preprocessed images
preprocessed_normal_folder = "new_preprocessed_clahe/normal/"
preprocessed_polyp_folder = "new_preprocessed_clahe/polyps/"
preprocessed_esophagitis_folder = "new_preprocessed_clahe/esophagitis/"
preprocessed_ulcerative_colitis_folder = "new_preprocessed_clahe/ulcerative-colitis/"

preprocessed_dirs = [preprocessed_normal_folder, preprocessed_polyp_folder, preprocessed_esophagitis_folder, preprocessed_ulcerative_colitis_folder]

# Load preprocessed images and labels
X, y = load_data(preprocessed_dirs)

# Data preprocessing
X = X.astype('float32') / 255.0
y = to_categorical(y, num_classes=4)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Loading the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Adding custom dense layers for multi-class classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Output layer with 4 units for 4 classes

# Combining the base model with custom dense layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freezing layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
################################################################

with tf.device('/cpu:0'):
   X_train = tf.convert_to_tensor(X_train, np.float32)
   y_train = tf.convert_to_tensor(y_train, np.float32)

################################################################

In [ ]:
# Start time
start_time = time.time()

# Training the model
model.fit(X_train, y_train, epochs = 10, batch_size = 8, validation_data = (X_test, y_test))

# End time
end_time = time.time()

# Duration of training
training_duration = end_time - start_time
print(f"Training took {round(training_duration, 2)} seconds.")
training_time_minutes = training_duration/60
training_time_hours = training_time_minutes/60
print(f"Training took {round(training_time_minutes, 2)} minutes.")
print(f"Training took {round(training_time_hours, 2)} hours.")

In [ ]:
# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

In [ ]:
# Saving the model
model.save('saved_models/v3_inceptionv3_clahe/')

In [7]:
from keras.models import load_model

# Load the saved model
loaded_model = load_model('saved_models/v3_inceptionv3_clahe/')

# Model evaluation
loss, accuracy = loaded_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

22/22 [==============================] - 17s 341ms/step - loss: 0.2752 - accuracy: 0.9171
Test Loss: 0.2752058506011963, Test Accuracy: 0.9171428680419922


In [8]:
def plot_history(history, model_name):
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(model_name + ' Model')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy / Loss')
    plt.legend()
    plt.show()

In [9]:
#plot_history(inception_history, 'InceptionV3')
#plot_history(resnet_history, 'ResNet50V2')
plot_history(loaded_model, 'ResNet50V2')

TypeError: 'History' object is not subscriptable

In [ ]:
Epoch 1/10
350/350 [==============================] - 59s 126ms/step - loss: 0.5645 - accuracy: 0.8011 - val_loss: 0.3416 - val_accuracy: 0.8857
Epoch 2/10
350/350 [==============================] - 40s 114ms/step - loss: 0.2982 - accuracy: 0.8779 - val_loss: 0.2673 - val_accuracy: 0.8971
Epoch 3/10
350/350 [==============================] - 41s 117ms/step - loss: 0.2522 - accuracy: 0.8986 - val_loss: 0.2821 - val_accuracy: 0.9000
Epoch 4/10
350/350 [==============================] - 40s 114ms/step - loss: 0.2233 - accuracy: 0.9136 - val_loss: 0.2326 - val_accuracy: 0.9114
Epoch 5/10
350/350 [==============================] - 40s 114ms/step - loss: 0.1982 - accuracy: 0.9229 - val_loss: 0.2134 - val_accuracy: 0.9157
Epoch 6/10
350/350 [==============================] - 40s 114ms/step - loss: 0.1783 - accuracy: 0.9268 - val_loss: 0.2188 - val_accuracy: 0.9229
Epoch 7/10
350/350 [==============================] - 40s 113ms/step - loss: 0.1600 - accuracy: 0.9393 - val_loss: 0.1979 - val_accuracy: 0.9314
Epoch 8/10
350/350 [==============================] - 40s 114ms/step - loss: 0.1508 - accuracy: 0.9350 - val_loss: 0.1989 - val_accuracy: 0.9257
Epoch 9/10
350/350 [==============================] - 40s 114ms/step - loss: 0.1332 - accuracy: 0.9471 - val_loss: 0.1909 - val_accuracy: 0.9371
Epoch 10/10
350/350 [==============================] - 41s 116ms/step - loss: 0.1310 - accuracy: 0.9446 - val_loss: 0.1977 - val_accuracy: 0.9400
Training took 421.67 seconds.
Training took 7.03 minutes.
Training took 0.12 hours.

In [ ]:
Epoch 1/10
350/350 [==============================] - 41s 115ms/step - loss: 0.1292 - accuracy: 0.9500 - val_loss: 0.2548 - val_accuracy: 0.9086
Epoch 2/10
350/350 [==============================] - 40s 114ms/step - loss: 0.1138 - accuracy: 0.9536 - val_loss: 0.2756 - val_accuracy: 0.9086
Epoch 3/10
350/350 [==============================] - 41s 117ms/step - loss: 0.0903 - accuracy: 0.9625 - val_loss: 0.2256 - val_accuracy: 0.9229
Epoch 4/10
350/350 [==============================] - 40s 114ms/step - loss: 0.0852 - accuracy: 0.9632 - val_loss: 0.2679 - val_accuracy: 0.9243
Epoch 5/10
350/350 [==============================] - 40s 115ms/step - loss: 0.1046 - accuracy: 0.9564 - val_loss: 0.2315 - val_accuracy: 0.9286
Epoch 6/10
350/350 [==============================] - 41s 118ms/step - loss: 0.0926 - accuracy: 0.9579 - val_loss: 0.2367 - val_accuracy: 0.9157
Epoch 7/10
350/350 [==============================] - 40s 114ms/step - loss: 0.0534 - accuracy: 0.9818 - val_loss: 0.2594 - val_accuracy: 0.9229
Epoch 8/10
350/350 [==============================] - 41s 117ms/step - loss: 0.0581 - accuracy: 0.9771 - val_loss: 0.3225 - val_accuracy: 0.9057
Epoch 9/10
350/350 [==============================] - 40s 114ms/step - loss: 0.0761 - accuracy: 0.9704 - val_loss: 0.2996 - val_accuracy: 0.9186
Epoch 10/10
350/350 [==============================] - 40s 114ms/step - loss: 0.0863 - accuracy: 0.9696 - val_loss: 0.2752 - val_accuracy: 0.9171
Training took 403.58 seconds.
Training took 6.73 minutes.
Training took 0.11 hours.